In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import os
from math import sqrt as sqrt
import numpy as np
import itertools
from torch.autograd import Function
import torch.nn.init as init
from ssd_project.model import ssd
from ssd_project.functions.detection import *
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
#color detection of red and green color in image
#necessary libraries 
import matplotlib.patches as patches
from imutils import paths
import argparse
import imutils
import os.path
from cv2 import cv2

# filter with building detection model (breif select)

In [2]:
os.chdir(r"C:/Users/Xiaorang/Desktop/Project/SSD_FacadeParsing/")

In [3]:
best_model = torch.load("./saved_models/Best_model_ssd300.pth.tar")
model = ssd.build_ssd(num_classes = 4)
model.load_state_dict(best_model["model_state_dict"])
device = "cuda"
model = model.to(device)

epochs_trained = best_model["epoch"]
best_avg_loss = best_model["loss"]
t_loss_bvals = best_model["training_losses_batch_values"]
t_loss_bavgs = best_model["training_losses_batch_avgs"]
v_loss_bvals = best_model["validation_losses_batch_values"]
v_loss_bavgs = best_model["validation_losses_batch_avgs"]

In [4]:
def return_num_img(img_path):
    arr = img_path.split("/")
    name = arr[len(arr)-1]
    arr = name.split("_")
    num = arr[len(arr)-1].split(".")[0]
    
    return num

In [6]:
def compare(position_window,position_building):
    """
    If the position of window lay outside the building reture False, otherwise True
    """
    if((position_window[0]>position_building[0])&(position_window[2]<position_building[2])&(position_window[3]<position_building[3])&(position_window[1]>position_building[1])):
        return True #in the building
    return False

In [13]:
path_to_save_good = "./data/prediction_good/"
path_to_save_bad = "./data/prediction_bad/"
file_good = "./data/prediction_good/ratios_good.csv"
file_bad = "./data/prediction_bad/ratios_bad.csv"
imgs = glob.glob("./data/test_images/*")
imgs.sort()
if os.path.exists(file_good):
    locFile_good = open(file_good,"a")
else:
    locFile_good = open(file_good,"w+")
    locFile_good.write("Names, Ratios\n")
if os.path.exists(file_bad):
    locFile_bad = open(file_bad,"a")
else:
    locFile_bad = open(file_bad,"w+")
    locFile_bad.write("Names, Ratios\n")
for i, img in enumerate(imgs):
    
    pred_img, bboxes, labels, scores = predict_objects(model, img, min_score=0.2, max_overlap = 0.01, top_k=200)
    annotated_img = FT.to_pil_image(draw_detected_objects(img, bboxes, labels, scores))
    num = return_num_img(img)
    name = "annotated_img_" + num + ".png"
    path_img_good = path_to_save_good + name
    path_img_bad = path_to_save_bad + name

    # Windows Ratio calculation 
    building_area = (bboxes[0][2] - bboxes[0][0])*(bboxes[0][3] - bboxes[0][1])
    position_building = [bboxes[0][0],bboxes[0][1],bboxes[0][2],bboxes[0][3]]
    windows_area = 0
    flag = True # assume it in the building
    for j in range(1,len(bboxes)):
        windows_area = windows_area + (bboxes[j][2] - bboxes[j][0])*(bboxes[j][3] - bboxes[j][1])
        position_window = [bboxes[j][0],bboxes[j][1],bboxes[j][2],bboxes[j][3]]
        flag = flag&compare(position_window,position_building)
#     if(flag == False):
#         print('The window outside the building')
    ratio = windows_area/building_area
    #if ((ratio>0.025)&(ratio<0.85)&flag):
    if ((ratio>0.025)&(ratio<0.85)): 
        pred_img.save(path_img_good, "PNG")
        locFile_good.write(name+","+str(float(ratio.detach().numpy()))+'\n')
    else:
        pred_img.save(path_img_bad, "PNG")
        locFile_bad.write(name+","+str(float(ratio.detach().numpy()))+'\n')
locFile_bad.close()
locFile_good.close()

KeyboardInterrupt: 

# filter with green ratio

In [14]:
def green_area(image):
    green = np.uint8([[[0, 255, 0]]])  #green color
    hsvGreen = cv2.cvtColor(green, cv2.COLOR_BGR2HSV) #hsv value of green color 
    lowerLimit = hsvGreen[0][0][0] - 20, 40, 40  # range of green color lower limit and upper limit
    upperLimit = hsvGreen[0][0][0] + 10, 255, 255
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) #convert the image into hsv
    lg = np.array(lowerLimit) #range of green color
    ug = np.array(upperLimit)

    green_mask = cv2.inRange(hsv, lg, ug) #green masked image    
    contours, hiera = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    area = 0
    for i in contours:
        area += cv2.contourArea(i)
    return area

In [18]:
imgPaths = sorted(list(paths.list_images("./data/prediction_good")))
path_to_save_bad = "./data/detect_green_removed/"
path_to_save_good = "./data/detect_green_retained/"
for imgPath in imgPaths:
    image = cv2.imread(imgPath) #load image 
    area_green = green_area(image)
    total_area = image.shape[0] * image.shape[1]
    ratio_green = area_green/ total_area
    #print(ratio_green)
    
    # Store the images with green_detected
    if ratio_green > 0.15:
        num = return_num_img(imgPath)
        name = "green_img_" + num + ".png"
        path_img_bad = path_to_save_bad + name
        cv2.imwrite(path_img_bad, image)
    else:
        num = return_num_img(imgPath)
        name = "green_img_" + num + ".png"
        path_img_good = path_to_save_good + name
        cv2.imwrite(path_img_good, image)

# filter cars with original SSD

In [19]:
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
ssd_model.to('cuda')
ssd_model.eval()

Using cache found in C:\Users\Xiaorang/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\Xiaorang/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [20]:
from imutils import paths
import cv2
imgPath = sorted(list(paths.list_images("./data/detect_green_retained")))
path_to_save_good = "./data/detect_car_retained/"
path_to_save_bad = "./data/detect_car_removed/"

while(imgPath):
    # Grab data in a group of three
    if len(imgPath)<3:
        uris = []
        for i in range(len(imgPath)):
            uris.append(imgPath.pop(0))
    else:
        uris = imgPath[0:3]
        imgPath = imgPath[3:]
    
    # Detect car
    try:
        images = [cv2.imread(k) for k in uris]
        inputs = [utils.prepare_input(uri) for uri in uris]
        tensor = utils.prepare_tensor(inputs, precision == 'fp16')
        detections_batch = ssd_model(tensor)
        results_per_input = utils.decode_results(detections_batch)
        best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]
        classes_to_labels = utils.get_coco_object_dictionary()
    except:
        continue

    for image_idx in range(len(best_results_per_input)):
        # Show original, denormalized image...
        #image = inputs[image_idx]
        image = images[image_idx]
        # ...with detections
        bboxes, classes, confidences = results_per_input[image_idx]
        area = 0
        confidence = 0
        if(3 in classes):
            area = 0
            for idx in np.where(classes==3)[0]:
                idx =int(idx)
                left, bot, right, top = bboxes[idx]
                area = area + (right - left)*(top - bot)
                confidence = max(confidence,confidences[idx])
#             print(area)
#             print(confidence)
        if (area>0.09) & (confidence>0.7):
            #image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
            num = return_num_img(uris[image_idx])
            name = "car_img_" + num + ".png"
            path_img_bad = path_to_save_bad + name
            cv2.imwrite(path_img_bad, image)
        else:
            #image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
            num = return_num_img(uris[image_idx])
            name = "car_img_" + num + ".png"
            path_img_good = path_to_save_good + name
            cv2.imwrite(path_img_good, image)

# Using the building detection model on filtered dataset

In [21]:
imgs = glob.glob("./data/detect_car_retained/*")
imgs.sort()
path_to_save = "./data/filtered_data/"
for i, img in enumerate(imgs):
    
    pred_img, bboxes, labels, scores = predict_objects(model, img, min_score=0.2, max_overlap = 0.01, top_k=200)
    annotated_img = FT.to_pil_image(draw_detected_objects(img, bboxes, labels, scores))
    num = return_num_img(img)
    
    path_img = path_to_save + "annotated_img_" + num + ".png"
    
    annotated_img.save(path_img, "PNG")